**Packages**

In [1]:
import pandas as pd
import re
import numpy as np
from openai import OpenAI
import os
from dotenv import load_dotenv

**Helpers imports**

In [2]:
from helpers.models import adjusted_r2, run_xgboost_cv
from helpers.data_extraction import extract_section_7, load_automobile_df
from helpers.llm_helpers import get_schema_text_for_llm, make_summary
from helpers.general_helpers import get_next_run_idx, get_interesting_idxs

Parameters api key and base url

In [3]:
load_dotenv()
api_key = os.getenv("key")
base_url = os.getenv("api")

Get the dataset

In [4]:
data_path = "automobile/imports-85.data"
names_path = "automobile/imports-85.names"

In [5]:
df, informations = load_automobile_df(data_path, names_path)

In [6]:
df.head()

,symboling,normalized-losses,make,fuel-type,aspiration,num-of-doors,body-style,drive-wheels,engine-location,wheel-base,...,engine-size,fuel-system,bore,stroke,compression-ratio,horsepower,peak-rpm,city-mpg,highway-mpg,price
0,3,NaN,alfa-romero,gas,std,two,convertible,rwd,front,88.6,...,130,mpfi,3.47,2.68,9.0,111.0,5000.0,21,27,13495.0
1,3,NaN,alfa-romero,gas,std,two,convertible,rwd,front,88.6,...,130,mpfi,3.47,2.68,9.0,111.0,5000.0,21,27,16500.0
2,1,NaN,alfa-romero,gas,std,two,hatchback,rwd,front,94.5,...,152,mpfi,2.68,3.47,9.0,154.0,5000.0,19,26,16500.0
3,2,164.0,audi,gas,std,four,sedan,fwd,front,99.8,...,109,mpfi,3.19,3.40,10.0,102.0,5500.0,24,30,13950.0
4,2,164.0,audi,gas,std,four,sedan,4wd,front,99.4,...,136,mpfi,3.19,3.40,8.0,115.0,5500.0,18,22,17450.0


In [7]:
informations

{'symboling': '-3, -2, -1, 0, 1, 2, 3.',
 'normalized-losses': 'continuous from 65 to 256.',
 'make': 'alfa-romero, audi, bmw, chevrolet, dodge, honda, isuzu, jaguar, mazda, mercedes-benz, mercury, mitsubishi, nissan, peugot, plymouth, porsche, renault, saab, subaru, toyota, volkswagen, volvo',
 'fuel-type': 'diesel, gas.',
 'aspiration': 'std, turbo.',
 'num-of-doors': 'four, two.',
 'body-style': 'hardtop, wagon, sedan, hatchback, convertible.',
 'drive-wheels': '4wd, fwd, rwd.',
 'engine-location': 'front, rear.',
 'wheel-base': 'continuous from 86.6 120.9.',
 'length': 'continuous from 141.1 to 208.1.',
 'width': 'continuous from 60.3 to 72.3.',
 'height': 'continuous from 47.8 to 59.8.',
 'curb-weight': 'continuous from 1488 to 4066.',
 'engine-type': 'dohc, dohcv, l, ohc, ohcf, ohcv, rotor.',
 'num-of-cylinders': 'eight, five, four, six, three, twelve, two.',
 'engine-size': 'continuous from 61 to 326.',
 'fuel-system': '1bbl, 2bbl, 4bbl, idi, mfi, mpfi, spdi, spfi.',
 'bore': 'c

In [8]:
print(df.dtypes)

symboling              int64
normalized-losses    float64
make                  object
fuel-type             object
aspiration            object
num-of-doors          object
body-style            object
drive-wheels          object
engine-location       object
wheel-base           float64
length               float64
width                float64
height               float64
curb-weight            int64
engine-type           object
num-of-cylinders      object
engine-size            int64
fuel-system           object
bore                 float64
stroke               float64
compression-ratio    float64
horsepower           float64
peak-rpm             float64
city-mpg               int64
highway-mpg            int64
price                float64
dtype: object


As the variables num-of-cylinders and num-of-doors have string values, despite having numerical meaning, it was decided to convert them to numerical values considering the values that are given.

In [9]:
# To confirm if the list of values matches the one that was given
print("Unique values of num-of-cylinders:", list(df['num-of-cylinders'].unique()))
print("Unique values of num-of-doors:", list(df['num-of-doors'].unique()))

Unique values of num-of-cylinders: ['four', 'six', 'five', 'three', 'twelve', 'two', 'eight']
Unique values of num-of-doors: ['two', 'four', nan]


In [10]:
door_map = {
    'four': 4, 
    'two': 2
}

cyl_map = {
    'eight': 8,
    'five': 5,
    'four': 4,
    'six': 6,
    'three': 3,
    'twelve': 12,
    'two': 2
}

# Convert to integer columns. I put Int64 because one variables has missing values.
df['num-of-doors'] = df['num-of-doors'].map(door_map).astype('Int64')
df['num-of-cylinders'] = df['num-of-cylinders'].map(cyl_map).astype('Int64')

Variables curb-weight and engine-size are "continuous" and not discrete, so their types were changed.

In [11]:
df['curb-weight'] = df['curb-weight'].astype('float64')
df['engine-size'] = df['engine-size'].astype('float64')

LLM Solution

***1. Data Analyst Agent***

In [12]:
client = OpenAI(
    base_url=base_url,
    api_key=api_key
)

In [13]:
def llm_analyze(summary, client, model="deepseek/deepseek-r1-0528:free",temperature=0.2):
    prompt = [
        {"role": "system", "content": (
            f"""You are an automotive data analyst agent.

            You will receive a summary of an automobile dataset, including:
            - A list of all columns, with their data types, attribute ranges or value examples, and missing value counts
            - Sample records
            - Key statistics (e.g. mean, std, min, max, skew, etc.) for numerical and categorical variables
            - Optional EDA findings (distribution shape, outlier notes, trends, correlations)

            Your responsabilities are:
            1. For each variable/column, explain what it likely measures, its units or categories, and its practical meaning in the context of vehicle data. Be explicit and do not group variables—analyze each one individually.
            2. Detect possible semantic relationships, physical or business meanings, and likely associations with other columns (e.g. which pairs have strong correlations, which affect price/risk, etc).
            3. Call out notable trends, strong correlations, anomalies, and outliers. Reference variable names and statistics explicitly.
            4. Suggest ways the dataset could be enriched: for each, mention relevant external datasets, additional columns, or real-world data sources (e.g. safety ratings, CO2 emissions, vehicle class, market info) that could be joined or mapped by column name.

            Format your answer as:
            - A numbered list where each variable is explained with name, meaning, type, and any key distribution or outlier notes.
            - A section on key trends, correlations and anomalies
            - A section with enrichment recommendations, referencing specific variable names

            Please, focus on schema interpretation (variable meanings), EDA, and actionable suggestions for deeper analysis.
            """
        )},
        {"role": "user", "content": f"Analyze this automotive dataset summary. Focus on variable meanings, EDA trends, and possible data enrichments.\n\nSUMMARY:\n{summary}"}
    ]
    
    response = client.chat.completions.create(model=model, messages=prompt, temperature=temperature)
    return response.choices[0].message.content


In [14]:
def run_llm_eda_agent(df, informations, client, n_records=3, eda_findings=None, 
                      model="deepseek/deepseek-r1-0528:free", 
                      temperature=0.2, 
                      out_dir="outputs/analyst_agent"):
    """
    Orchestrates building the summary, calling the LLM agent, and saving output.
    
    Input:
        df: pandas DataFrame
        informations: dict of variable names to attribute range/description
        client: OpenAI/OpenRouter SDK client
        n_records: int, number of samples to show
        eda_findings: str, optional,  EDA highlights that the data scientist think will be useful.
        model: str, LLM model name
        temperature: float, LLM creativity level
        out_dir: str, where to save the output
    Output:
        analysis (str): formatted LLM agent analysis
    """
    summary = make_summary(df, informations, n_records=n_records, eda_findings=eda_findings)
    print("\n=== Raw Data Summary sent to LLM ===\n")
    print(summary[:1500], "...\n")
    
    analysis = llm_analyze(summary, client, model=model, temperature=temperature)
    print("\n=== LLM Analysis ===\n")
    print(analysis)
    
    idx = get_next_run_idx(out_dir, agent_name="llm_analyst")
    txt_filename = os.path.join(out_dir, f"{idx}_llm_analyst.txt")
    with open(txt_filename, "w", encoding="utf-8") as f:
        f.write(analysis)
    return analysis

In [15]:
run_llm_eda_agent(df, informations, client)


=== Raw Data Summary sent to LLM ===

Rows: 205; Columns: 26

Column Attribute range:
- symboling (int64) | Missing: 0 | Attribute range: -3, -2, -1, 0, 1, 2, 3.
- normalized-losses (float64) | Missing: 41 | Attribute range: continuous from 65 to 256.
- make (object) | Missing: 0 | Attribute range: alfa-romero, audi, bmw, chevrolet, dodge, honda, isuzu, jaguar, mazda, mercedes-benz, mercury, mitsubishi, nissan, peugot, plymouth, porsche, renault, saab, subaru, toyota, volkswagen, volvo
- fuel-type (object) | Missing: 0 | Attribute range: diesel, gas.
- aspiration (object) | Missing: 0 | Attribute range: std, turbo.
- num-of-doors (Int64) | Missing: 2 | Attribute range: four, two.
- body-style (object) | Missing: 0 | Attribute range: hardtop, wagon, sedan, hatchback, convertible.
- drive-wheels (object) | Missing: 0 | Attribute range: 4wd, fwd, rwd.
- engine-location (object) | Missing: 0 | Attribute range: front, rear.
- wheel-base (float64) | Missing: 0 | Attribute range: continuous 

'### 1. Variable Analysis (Individual Breakdown)\n\n1. **symboling** (int64)  \n   - **Meaning**: Insurance risk rating (-3 = lowest risk, 3 = highest risk). Reflects historical claim data.  \n   - **Distribution**: Mean ≈ 0.83, slight positive skew (0.21). Most vehicles cluster at 0–2 (neutral to moderate risk).  \n\n2. **normalized-losses** (float64)  \n   - **Meaning**: Average insurance loss relative to other vehicles (higher = costlier claims). Unitless index.  \n   - **Missing**: 20% missing (41 rows). Range 65–256, mean 122. Moderate skew (0.77).  \n\n3. **make** (object)  \n   - **Meaning**: Vehicle manufacturer (e.g., Toyota, BMW).  \n   - **Distribution**: Toyota most frequent (32/205), Nissan second (18/205).  \n\n4. **fuel-type** (object)  \n   - **Meaning**: Engine fuel type (gas/diesel).  \n   - **Distribution**: Gas dominates (185/205 vs. 20 diesel).  \n\n5. **aspiration** (object)  \n   - **Meaning**: Air intake system (std = natural, turbo = forced induction).  \n   - 

Example of output:

### 1. Variable Analysis (Individual Breakdown)
1. **symboling** (int64):  
   - **Meaning**: Insurance risk rating (-3=low risk, 3=high risk). Reflects vehicle safety/claim likelihood.  
   - **Distribution**: Symmetric (skew=0.21), mode=0. Most vehicles are neutral/low-risk (mean=0.83).  

2. **normalized-losses** (float64):  
   - **Meaning**: Average insurance loss payments (normalized). Higher values indicate greater losses.  
   - **Units**: Arbitrary units (65–256).  
   - **Notes**: 20% missing (41 rows). Slight right-skew (skew=0.77), max=256 may indicate high-risk outliers.  

3. **make** (object):  
   - **Meaning**: Vehicle manufacturer (e.g., Toyota, BMW).  
   - **Categories**: 22 brands. Toyota most frequent (32/205), Nissan second (18).  

4. **fuel-type** (object):  
   - **Meaning**: Engine fuel type.  
   - **Categories**: Gas (185), diesel (20). Dominated by gas vehicles.  

5. **aspiration** (object):  
   - **Meaning**: Air intake system.  
   - **Categories**: Standard (168), turbocharged (37). Turbocharged engines are less common.  

6. **num-of-doors** (Int64):  
   - **Meaning**: Door count.  
   - **Categories**: Two (rare), four (common). 2 missing values. Mean=3.12 confirms four doors dominate.  

7. **body-style** (object):  
   - **Meaning**: Vehicle body design.  
   - **Categories**: Sedan (96), hatchback (70), convertible, hardtop, wagon. Sedans are most prevalent.  

8. **drive-wheels** (object):  
   - **Meaning**: Wheel-drive configuration.  
   - **Categories**: FWD (120), RWD (76), 4WD (9). Front-wheel drive dominates.  

9. **engine-location** (object):  
   - **Meaning**: Engine placement.  
   - **Categories**: Front (202), rear (3). Rear-engine vehicles are rare anomalies.  

10. **wheel-base** (float64):  
    - **Meaning**: Distance between front/rear axles (stability indicator).  
    - **Units**: Inches (86.6–120.9).  
    - **Distribution**: Right-skewed (skew=1.05), suggesting some longer-wheelbase outliers.  

11. **length** (float64):  
    - **Meaning**: Vehicle length.  
    - **Units**: Inches (141.1–208.1).  
    - **Distribution**: Symmetric (skew=0.16), median=173.2.  

12. **width** (float64):  
    - **Meaning**: Vehicle width.  
    - **Units**: Inches (60.3–72.3).  
    - **Distribution**: Right-skewed (skew=0.90), max=72.3 may indicate wide luxury/performance models.  

13. **height** (float64):  
    - **Meaning**: Vehicle height.  
    - **Units**: Inches (47.8–59.8).  
    - **Distribution**: Symmetric (skew=0.06), median=54.1.  

14. **curb-weight** (float64):  
    - **Meaning**: Vehicle weight without occupants.  
    - **Units**: Pounds (1488–4066).  
    - **Distribution**: Right-skewed (skew=0.68). Heavy vehicles (>3500 lbs) are potential outliers.  

15. **engine-type** (object):  
    - **Meaning**: Engine design.  
    - **Categories**: OHC (148), OHCF (15), others. Overhead-cam engines dominate.  

16. **num-of-cylinders** (Int64):  
    - **Meaning**: Engine cylinder count.  
    - **Categories**: Four-cylinder (mode=4), six, eight, twelve. Mean=4.38; twelve-cylinders are rare.  

17. **engine-size** (float64):  
    - **Meaning**: Engine displacement.  
    - **Units**: Cubic centimeters (61–326).  
    - **Distribution**: Highly right-skewed (skew=1.95). Max=326 (performance outliers).  

18. **fuel-system** (object):  
    - **Meaning**: Fuel delivery system.  
    - **Categories**: MPFI (94), 2BBL (66). Multi-point fuel injection is most common.  

19. **bore** (float64):  
    - **Meaning**: Cylinder bore diameter.  
    - **Units**: Inches (2.54–3.94).  
    - **Notes**: 4 missing values. Symmetric (skew=0.02).  

20. **stroke** (float64):  
    - **Meaning**: Piston stroke length.  
    - **Units**: Inches (2.07–4.17).  
    - **Notes**: 4 missing values. Left-skewed (skew=-0.68).  

21. **compression-ratio** (float64):  
    - **Meaning**: Engine compression efficiency.  
    - **Units**: Ratio (7–23).  
    - **Distribution**: Extreme right-skew (skew=2.61). Max=23 (diesel/high-performance outliers).  

22. **horsepower** (float64):  
    - **Meaning**: Engine power output.  
    - **Units**: Horsepower (48–288).  
    - **Notes**: 2 missing values. Right-skewed (skew=1.39); 288 HP is a high-performance outlier.  

23. **peak-rpm** (float64):  
    - **Meaning**: RPM at max horsepower.  
    - **Units**: Revolutions/minute (4150–6600).  
    - **Distribution**: Symmetric (skew=0.07), mode=5500 RPM.  

24. **city-mpg** (int64):  
    - **Meaning**: City fuel efficiency.  
    - **Units**: Miles/gallon (13–49).  
    - **Distribution**: Right-skewed (skew=0.66). Low-efficiency vehicles (<15 MPG) are outliers.  

25. **highway-mpg** (int64):  
    - **Meaning**: Highway fuel efficiency.  
    - **Units**: Miles/gallon (16–54).  
    - **Distribution**: Right-skewed (skew=0.54), similar to city-mpg.  

26. **price** (float64):  
    - **Meaning**: Vehicle retail price.  
    - **Units**: Dollars (5118–45400).  
    - **Notes**: 4 missing values. Highly right-skewed (skew=1.81). Max=$45,400 (luxury outliers).  

---

### 2. Key Trends, Correlations, and Anomalies  
- **Strong Correlations**:  
  - **Engine-size ↔ Horsepower** (expected): Larger engines produce more power (e.g., engine-size max=326 cc links to horsepower max=288).  
  - **Dimensions ↔ Weight**: Length, width, wheel-base, and curb-weight correlate (e.g., large vehicles like SUVs have high curb-weight).  
  - **MPG ↔ Performance**: Negative correlation between city/highway-mpg and horsepower/engine-size (high-power engines reduce efficiency).  
  - **Price ↔ Performance**: Luxury/performance vehicles (high horsepower, large engines) drive price skew (max=$45,400).  

- **Anomalies**:  
  - **Rear-engine vehicles**: Only 3 exist (vs. 202 front-engine), likely specialty cars (e.g., Porsche 911).  
  - **Compression-ratio=23**: Extreme outlier (typical range: 8–12); may indicate diesel engines or data errors.  
  - **Twelve-cylinder engines**: Rare (num-of-cylinders=12), associated with high price/horsepower.  
  - **Normalized-losses**: 20% missing; high values (max=256) may indicate high-risk vehicles.  

- **Trends**:  
  - Most vehicles are gas-powered, front-engine, FWD sedans with 4-cylinder OHC engines.  
  - Turbocharged (aspiration=turbo) and diesel (fuel-type=diesel) vehicles are minorities.  

---

### 3. Enrichment Recommendations  
1. **Safety Ratings (e.g., NHTSA/IIHS)**:  
   - Join via `make` and `body-style` to contextualize `symboling` (risk-rating). Explains why some vehicles have negative symboling.  

2. **CO2 Emissions Data (EPA)**:  
   - Augment `fuel-type` and `fuel-system` to analyze environmental impact vs. `city-mpg`/`highway-mpg`.  

3. **Vehicle Class (e.g., "SUV", "Sports Car")**:  
   - Derive from `body-style`, `curb-weight`, and `length` to segment analysis (e.g., luxury SUVs vs. compact sedans).  

4. **Market Data (e.g., Sales Volume, Year)**:  
   - Add `model-year` to track trends (e.g., turbo adoption over time). Sales data per `make` explains pricing anomalies.  

5. **Transmission Type**:  
   - Missing but critical for performance/efficiency analysis. Source from manufacturer databases.  

6. **Crash Test Scores**:  
   - Link to `normalized-losses` to validate insurance risk metrics.  

**Targeted Enrichment by Variable**:  
- For `price`: Add inflation-adjusted MSRP and used-car depreciation rates.  
- For `horsepower`/`engine-size`: Include torque ratings for performance context.  
- For `num-of-doors`: Map to vehicle segments (e.g., two-doors = sports cars).

***The suggestions in 2) of the data analyst agent’s output are in line with what was obtained in the notebook eda_classic_ml.ipynb. My own analysis was already complete, but this agent can be very useful for someone who wants to start with a baseline of what can be expected and which plots should be created to verify the claims made by the Agent Data Analyst.***

***2. FEATURE ENGINEERING AGENT***

I have two prompts: the second creates more difficult mathematical variables

In [16]:
prompt_header_1 = f"""prompt_header = 
You are an automotive feature engineering agent, working as part of a multi-agent analytics workflow for a car dataset.

You will receive:
- The current dataset schema (column names with attribute ranges), dtype, missing value counts, and a summary of key EDA findings/trends/outliers.

Your objectives are:
1. Propose new features by transforming/combine existing variables, or by suggesting realistic external enrichment.
2. For each suggestion, clearly specify:
    - Feature Name
    - Formula: Describe how to calculate or construct the feature, using column names/attribute ranges or plausible external columns.
    - Business/Domain Justification: Explain why this feature could have predictive or business value (use automotive logic: e.g., performance, safety, economics, etc).
    - Reasoning Chain:  Provide a step-by-step explanation, using EDA findings, variable statistics, or business logic, showing exactly why this feature should be selected and included in the modeling process (i.e., why it will add predictive value, interpretability, or business insight).
3. Document your output with a markdown table containing these four columns:
    | Feature Name | Formula/Recipe | Justification | Reasoning Chain |

Guidelines:
- Use only the columns listed in the schema and the EDA findings. Do NOT invent raw variables/columns.
- Enrichment/external columns must be plausible and documented ("external source: EPA", etc).
- Reference specific variable names and their attribute ranges as much as possible.
- Reason both like an ML feature engineer *and* an automotive business analyst.

Example output format: 
| Feature Name | Formula/Recipe | Justification| Reasoning Chain |
|--------------|----------------|--------------|-----------------|

Now, suggest as many well-justified, thoughtful new features as possible for both interpretability and predictive modeling, taking into account the summary:

"""

prompt_header_2 = f"""
You are an automotive feature engineering agent.

You will receive:
- A summary of the dataset schema: column names, attribute ranges, data types and missing value counts.

Your tasks are:
1. Purpose new features for predictive modeling and interpretability by combining, transforming, enriching the columns provided or by suggesting realistic external enrichment.
2. For each suggestion, specify:
    - Feature Name
    - Formula – Show how it is calculated (including mathematical formula or logic or code-like description referencing column names and attribute ranges)
    - Justification – Business or automotive domain reason for its value (e.g., why it matters for price, efficiency, safety, etc.)
    - Reasoning Chain – Step-by-step logic or EDA findings that motivate this feature (why it could improve prediction, segmentation, or insight)

Guidelines:
    - Use only the columns in the schema/EDA unless you explicitly recommend a realistic external enrichment (note the external source in justification).
    - Cite variable names and attribute ranges where helpful.
    - Suggest both classic (e.g., ratios, interactions, indicators) and creative features, but keep formulas practical—no fictitious data or overly complex recipes.

Output all suggestions as a markdown table:
| Feature Name | Formula        | Justification | Reasoning Chain |
|--------------|----------------|---------------|-----------------|

Then, in a short section below, summarize the main rationale for your key suggestions (e.g., which features link to performance, efficiency, market segmentation, or domain knowledge).
"""

prompt_tail = f"""
Schema and EDA summary follows:
"""

In [17]:
def llm_feature_engineering(summary, client, prompt_header, prompt_tail, model="deepseek/deepseek-r1-0528:free", 
                            temperature=0.3, out_dir="outputs/feature_agent"):
    prompt = prompt_header + prompt_tail + summary

    messages = [
        {"role": "system", "content": prompt},
        {"role": "user", "content": "Suggest as many well-justified new or engineered features as possible, for both interpretability and predictive modeling."}
    ]

    response = client.chat.completions.create(model=model, messages=messages, temperature=temperature)
    idx = get_next_run_idx(out_dir, agent_name="llm_feature")
    txt_filename = os.path.join(out_dir, f"{idx}_llm_feature.txt")
    output = response.choices[0].message.content
    
    with open(txt_filename, "w", encoding="utf-8") as f:
        f.write(output)
    return output
    

In [18]:
summary = make_summary(df, informations, n_records=3, eda_findings=None)
output = llm_feature_engineering(summary, client, prompt_header_1, prompt_tail)
print(output)

Based on the dataset schema and EDA findings, here are 12 well-justified engineered features for automotive price prediction and business insights:

| Feature Name | Formula/Recipe | Justification | Reasoning Chain |
|--------------|----------------|--------------|-----------------|
| **Power-to-Weight Ratio** | `horsepower / curb-weight` | Direct performance indicator influencing price premium | Higher ratios correlate with sports/luxury vehicles. EDA shows horsepower (48-288) and curb-weight (1488-4066) vary significantly. This ratio quantifies performance efficiency. |
| **Displacement Volume** | `π * (bore/2)^2 * stroke * num-of-cylinders` | Fundamental engine size metric affecting performance | Combines bore (2.54-3.94), stroke (2.07-4.17), and cylinders. Missing bore/stroke (4 NAs) require imputation. Larger displacement typically increases manufacturing cost and price. |
| **Fuel Efficiency Index** | `(highway-mpg * 0.7) + (city-mpg * 0.3)` | Weighted efficiency metric for real-

In [19]:
summary = make_summary(df, informations, n_records=3, eda_findings=None)
output = llm_feature_engineering(summary, client, prompt_header_2, prompt_tail)
print(output)

### Engineered Feature Suggestions

| Feature Name             | Formula                                                                 | Justification                                                                 | Reasoning Chain |
|--------------------------|-------------------------------------------------------------------------|-------------------------------------------------------------------------------|-----------------|
| **power_to_weight**      | `horsepower / curb-weight`                                              | Higher ratios indicate better acceleration and performance, influencing price and desirability. | Performance is a key factor in automotive pricing. Combining engine output (horsepower) with vehicle mass (curb-weight) creates a direct performance metric. |
| **displacement_per_cylinder** | `engine_size / (num_of_cylinders mapped to integer: {"two":2, "three":3, "four":4, "five":5, "six":6, "eight":8, "twelve":12})` | Reveals engine design efficiency. Larg

In [20]:
summary = make_summary(df, informations, n_records=3, eda_findings=None)
output = llm_feature_engineering(summary, client, prompt_header_2, prompt_tail)
print(output)

### Engineered Feature Suggestions

| Feature Name             | Formula                                                                 | Justification                                                                 | Reasoning Chain                                                                                                                                 |
|--------------------------|-------------------------------------------------------------------------|-------------------------------------------------------------------------------|-------------------------------------------------------------------------------------------------------------------------------------------------|
| **power_to_weight_ratio** | `horsepower / curb-weight`                                              | Higher ratios indicate better performance and acceleration, directly influencing desirability and price. | Performance-oriented buyers prioritize power efficiency. Combining engine output (horsepower) w

**3. Training XGBOOST with all features and with the new features suggested by the feature_engineering agent (I use the first set of features with header_1)**

In [21]:
features = [col for col in df.columns if col != 'price']
xgboost_results_original = run_xgboost_cv(df, features, False)

Fitting 5 folds for each of 96 candidates, totalling 480 fits
Best parameters: {'learning_rate': 0.01, 'max_depth': 3, 'n_estimators': 400, 'subsample': 1}
Best CV RMSE: 2616.435608016184
Test RMSE: 2724.65
Test MAE: 1810.04
Test R²: 0.905
Adjusted R²: 0.748

Top Feature Importances:
make           0.280320
engine-size    0.167140
engine-type    0.133444
curb-weight    0.121030
highway-mpg    0.055936
fuel-system    0.036141
horsepower     0.031452
city-mpg       0.028791
width          0.024451
body-style     0.023890
dtype: float32


features suggested (from one of the runs):

| Feature Name             | Formula/Recipe                                                                 | Justification                                                                 | Reasoning Chain                                                                                                                                                                                                 |
|--------------------------|-------------------------------------------------------------------------------|-------------------------------------------------------------------------------|-----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------|
| **Power-to-Weight**      | `horsepower / curb-weight`                                                    | Performance metric influencing acceleration and driving dynamics              | Higher ratios indicate better performance. EDA shows horsepower (48-288) and curb-weight (1488-4066) vary significantly. Sports cars (e.g., Porsche) will standout. Predictive for premium pricing.             |
| **Fuel_Efficiency_Index**| `(city_mpg * 0.55) + (highway_mpg * 0.45)`                                    | Real-world fuel economy reflecting operating costs                            | EPA uses 55% city/45% highway weighting. EDA shows mpg ranges (13-49 city, 16-54 highway). Critical for economy cars (e.g., Honda) vs. performance models.                                                    |
| **Aerodynamic_Ratio**    | `width / height`                                                              | Stability and drag coefficient proxy                                          | Lower ratios improve handling and fuel efficiency. Width (60.3-72.3) and height (47.8-59.8) vary. Sports cars typically have lower ratios (wider & flatter). Correlates with premium branding.                  |
| **Engine_Displacement**  | `π * (bore/2)^2 * stroke * num-of-cylinders`                                  | True engine size (cc) reflecting power potential                             | Combines bore (2.54-3.94), stroke (2.07-4.17), and cylinders. More accurate than engine-size alone. High-displacement engines (e.g., Jaguar V12) command price premiums.                                        |
| **Turbo_Premium**        | `horsepower if aspiration=='turbo' else 0`                                    | Value of forced induction technology                                          | Turbocharged engines (37 cases) produce more power/size. EDA shows turbo models have 23% avg horsepower boost. Captures technology surcharge.                                                                  |
| **Luxury_Brand**         | `1 if make in ['bmw','mercedes-benz','jaguar','porsche','audi','volvo'] else 0`| Brand equity premium                                                          | Top brands (Toyota=32, Nissan=18) dominate volume, but luxury brands command 42% avg price premium. Critical for price segmentation.                                                                           |
| **Cylinder_Class**       | `'low' if cylinders<=4, 'med' if 5-6, 'high' if >=8`                          | Engine sophistication tier                                                    | Cylinders range from 2-12 (skewed: 75% ≤4). High-cylinder engines (e.g., V8) correlate with luxury/performance. Handles non-linearity better than raw count.                                                  |
| **Safety_Score**         | `-1 * symboling`                                                              | Inverse risk rating (higher = safer)                                          | Symboling (-3 to 3) is insurance risk rating. EDA shows skew (mean 0.83). Safer cars (e.g., Volvo) have negative values. Directly impacts consumer willingness-to-pay.                                         |
| **Footprint**            | `wheel_base * track_width` (track_width = avg wheel position ≈ width)         | Vehicle platform size classification                                          | Wheelbase (86.6-120.9) and width determine interior space. Larger footprints indicate premium segments (sedans/SUVs). Critical for EPA size classes.                                                           |
| **FuelSystem_Tech**      | `1 if fuel_system in ['mpfi','mfi','spdi'] else 0`                            | Advanced injection technology indicator                                       | MPFI systems (94 cases) improve efficiency/power. Basic systems (1bbl/2bbl) correlate with economy models (e.g., Plymouth). Captures engineering quality.                                                     |
| **HighPerf_Indicator**   | `(compression_ratio > 10) & (aspiration == 'turbo')`                          | Performance engine signature                                                  | High compression (7-23, mean 10.14) + turbo indicates tuned engines. EDA shows only 18 such vehicles (e.g., Porsche). Strong premium predictor.                                                                |
| **Weight_Distribution**  | `1 if (engine_location=='rear') else (wheel_base/length)`                     | Handling characteristics                                                     | Rear-engine (3 cases) or front-engine balance. Ratio (0.52-0.59) affects stability. Sports cars (e.g., 911) have unique profiles impacting value.                                                              |
| **Maintenance_Risk**     | `normalized_losses * (num_of_cylinders/4)`                                    | Projected repair cost proxy                                                   | Losses (65-256) correlate with repair complexity. Adjusted by cylinder count (more cylinders = higher costs). Accounts for 20% missing data patterns.                                                           |
| **Efficiency_Penalty**   | `engine_size / city_mpg`                                                      | Fuel consumption per displacement                                             | Measures efficiency tradeoffs. High values indicate "gas guzzlers" (e.g., Chevrolet 326ci engine, 13 mpg). Explains price depreciation for inefficient models.                                                |
| **Drivetrain_Premium**   | `1 if drive_wheels == 'rwd' or '4wd' else 0`                                  | Performance/utility capability surcharge                                      | RWD/4WD (76+ cars) have 28% avg price premium over FWD. Critical for trucks/performance models. EDA shows FWD dominates (120 cases) but commands lower prices.                                                  |

In [22]:
df_feat = df.copy()

In [23]:
df_feat['power_to_weight'] = df_feat['horsepower'] / df_feat['curb-weight']

df_feat['fuel_efficiency_index'] = (df_feat['city-mpg'] * 0.55) + (df_feat['highway-mpg'] * 0.45)

df_feat['aero_ratio'] = df_feat['width'] / df_feat['height']

df_feat['engine_displacement'] = (
    np.pi * (df_feat['bore'] / 2) ** 2 * df_feat['stroke'] * df_feat['num-of-cylinders']
)

df_feat['turbo_premium'] = np.where(df_feat['aspiration'] == 'turbo', df_feat['horsepower'], 0)

luxury_brands = ['bmw', 'mercedes-benz', 'jaguar', 'porsche', 'audi', 'volvo']
df_feat['luxury_brand'] = df_feat['make'].isin(luxury_brands).astype(int)

def cyl_class(n):
    if n <= 4:
        return 'low'
    elif 5 <= n <= 6:
        return 'med'
    elif n >= 8:
        return 'high'
    else:
        return np.nan
df_feat['cylinder_class'] = df_feat['num-of-cylinders'].apply(cyl_class)

df_feat['safety_score'] = -1 * df_feat['symboling']

df_feat['footprint'] = df_feat['wheel-base'] * df_feat['width']

advanced_fs = ['mpfi', 'mfi', 'spdi']
df_feat['fuelsystem_tech'] = df_feat['fuel-system'].isin(advanced_fs).astype(int)

df_feat['highperf_indicator'] = ((df_feat['compression-ratio'] > 10) & (df_feat['aspiration'] == 'turbo')).astype(int)

df_feat['weight_distribution'] = np.where(
    df_feat['engine-location'] == 'rear',
    1,
    df_feat['wheel-base'] / df_feat['length']
)

df_feat['maintenance_risk'] = df_feat['normalized-losses'] * (df_feat['num-of-cylinders'] / 4)

df_feat['efficiency_penalty'] = df_feat['engine-size'] / df_feat['city-mpg']

df_feat['drivetrain_premium'] = df_feat['drive-wheels'].isin(['rwd', '4wd']).astype(int)

In [24]:
features = [col for col in df_feat.columns if col != 'price']
xgboost_results_llm = run_xgboost_cv(df_feat, features, False)

Fitting 5 folds for each of 96 candidates, totalling 480 fits
Best parameters: {'learning_rate': 0.03, 'max_depth': 3, 'n_estimators': 400, 'subsample': 0.8}
Best CV RMSE: 2668.468057110544
Test RMSE: 2825.62
Test MAE: 1769.54
Test R²: 0.898
Adjusted R²: nan

Top Feature Importances:
engine-size              0.265128
efficiency_penalty       0.121094
make                     0.085222
fuel_efficiency_index    0.080425
curb-weight              0.077970
highway-mpg              0.072301
num-of-cylinders         0.049024
engine-type              0.037614
fuel-system              0.030400
drive-wheels             0.028172
dtype: float32


The RMSE increase a bit with this new features, but the value of R squared increase. Let's try only using the top 10

In [25]:
features_top10 = [
    'engine-size',
    'efficiency_penalty',
    'make',
    'fuel_efficiency_index',
    'curb-weight',
    'highway-mpg',
    'num-of-cylinders',
    'engine-type',
    'fuel-system',
    'drive-wheels'
]

In [26]:
features = [col for col in df_feat.columns if col != 'price']
xgboost_results_llm = run_xgboost_cv(df_feat, features_top10, False)

Fitting 5 folds for each of 96 candidates, totalling 480 fits
Best parameters: {'learning_rate': 0.1, 'max_depth': 3, 'n_estimators': 400, 'subsample': 0.8}
Best CV RMSE: 2657.775772939511
Test RMSE: 2627.07
Test MAE: 1736.67
Test R²: 0.912
Adjusted R²: 0.883

Top Feature Importances:
highway-mpg              0.344025
make                     0.169481
engine-size              0.137727
engine-type              0.092489
fuel_efficiency_index    0.066583
fuel-system              0.063595
curb-weight              0.062157
efficiency_penalty       0.025538
drive-wheels             0.022142
num-of-cylinders         0.016264
dtype: float32


So, with this top 10 and using features recommended by the LLM solution, we got a lower value of RMSE and a pretty high value of R squared (91.2%)

***4. Explainer Agent***

In [27]:
def explain_prediction_llm(row, y_true, y_pred, importances, rmse, r2, cv_rmse, client,
                           ml_model="XGBoost", model="deepseek/deepseek-r1-0528:free",
                           txt_filename="llm_explanation.txt", temperature=0.2
):
    
    top_features = importances.head(5).index.tolist()
    feature_vals = "\n".join([f"- **{feat}**: {row[feat]}" for feat in top_features])
    prompt = f"""
    You are an AI assistant for explaining car price predictions to non-technical stakeholders

    A machine learning model ({ml_model}) was trained to predict vehicle prices based on technical and categorical features.
    Here are its key performance metrics:
    - Test RMSE: ${rmse:.2f} —   the typical error in predicted prices
    - Test R²: ${r2:.3f} — explanatory power on unseen data
    - CV RMSE: ${cv_rmse:.2f} — cross-validated training error

    Below is one car's prediction:
    - Actual price: ${y_true:.2f}
    - Predicted price: ${y_pred:.2f}

    Key features for this car:  
    {feature_vals}

    Top overall features driving model predictions:  
    {top_features}

    Your task (for non-technical stakeholders):

    - Write a clear, plain-language explanation of why the model predicted this car’s price as it did.  
        - Reference the most important features and their values for this specific car.
        - Highlight which features make it expensive or cheap.
        - Mention if the prediction is likely to be especially reliable or uncertain (for example, maybe the car has rare features or brand).
    - Briefly explain what the overall performance metrics (RMSE, R²) mean for buyers, sellers, or decision makers.

    Don’t use technical terms unless they are explained above. Always relate your explanation to real-world or business ideas.
    """
    messages = [
        {"role": "system", "content": "You are a helpful AI analytics explainer."},
        {"role": "user", "content": prompt}
    ]
    response = client.chat.completions.create(model=model, messages=messages, temperature=temperature)

    output = response.choices[0].message.content
    with open(txt_filename, "w", encoding="utf-8") as f:
        f.write(output)
    return output

In [28]:
y_test = xgboost_results_llm['y_test']
y_pred = xgboost_results_llm['y_pred']
importances = xgboost_results_llm['importances']
X_test = xgboost_results_llm['X_test']
cv_rmse = xgboost_results_llm['cv_rmse']
rmse = xgboost_results_llm['rmse']
r2 = xgboost_results_llm['r2']


In [29]:
interesting_idxs = get_interesting_idxs(y_test, y_pred)

In [30]:
def explain_interesting_cases(
    interesting_idxs, X_test, y_test, y_pred,
    importances, rmse, r2, cv_rmse, client, ml_model, next_idx,
    out_dir="outputs/explanation_agent",
    temperature=0.2
):
    """
    Function that runs all interesing indices and call explain_prediction_llm for each case, using a given next_idx.

    Inputs:
    interesting_idxs : list of int
        Indices to explain in the test set.
    X_test : pandas DataFrame
        Test set features.
    y_test : pandas Series or array
        True test values (must line up with X_test index).
    y_pred : array
        Corresponding model predictions.
    importances : any
        Feature importance data to pass to explainer.
    rmse, r2, cv_rmse : float
        Metrics for the explainer.
    client : object
        LLM client object.
    ml_model : str
        Model name string.
    next_idx : str
        The zero-padded index string for this batch of explanations.
    out_dir : str
        Directory to save explanations.
    temperature : float
        LLM temperature setting.

    Output:
    outputs: dict
        Dictionary mapping idx to explanation string.
    """
    outputs = {}
    for idx in interesting_idxs:
        row = X_test.iloc[idx]
        y_true = y_test.iloc[idx]
        y_hat = y_pred[idx]
        txt_filename = os.path.join(out_dir, f"{next_idx}_llm_explanation_{idx}.txt")
        output = explain_prediction_llm(
            row=row,
            y_true=y_true,
            y_pred=y_hat,
            importances=importances,
            rmse=rmse,
            r2=r2,
            cv_rmse=cv_rmse,
            client=client,
            ml_model=ml_model,
            txt_filename=txt_filename,
            temperature=temperature
        )
        outputs[idx] = output
        print(f"\n=== Explanation for test row {idx} ===\n")
        print(output)
    return outputs

In [31]:
out_dir = "outputs/explanation_agent"
next_idx = get_next_run_idx(out_dir, agent_name="llm_explanation")
ml_model = "XGBoost"

In [32]:
explain_interesting_cases(interesting_idxs, X_test, y_test, y_pred, importances, rmse, r2, cv_rmse, client, ml_model,next_idx)


=== Explanation for test row 15 ===

### Explanation for the Car's Predicted Price  
The model predicted this Toyota's price at **$6,384.95**, very close to its actual price of **$6,488**. Here’s why:  

#### Key Features Driving the Prediction:  
1. **High Fuel Efficiency**  
   - **Highway MPG: 38** (well above average)  
   - **Fuel Efficiency Index: 34.15** (also high)  
   These values signal strong fuel economy. While efficiency often *increases* a car’s value, this vehicle’s **small engine size** (next point) offsets that, making it more affordable.  

2. **Small Engine Size**  
   - **Engine Size: 92.0** (smaller than typical sedans)  
   Smaller engines usually mean lower power and **lower cost**. This is the #1 reason the prediction leans toward a moderate price.  

3. **Trusted Brand, Common Engine**  
   - **Make: Toyota** (reputable brand, holds value well)  
   - **Engine Type: OHC** (a common, reliable design)  
   Toyota’s reliability adds value, but the OHC engine is 

{15: '### Explanation for the Car\'s Predicted Price  \nThe model predicted this Toyota\'s price at **$6,384.95**, very close to its actual price of **$6,488**. Here’s why:  \n\n#### Key Features Driving the Prediction:  \n1. **High Fuel Efficiency**  \n   - **Highway MPG: 38** (well above average)  \n   - **Fuel Efficiency Index: 34.15** (also high)  \n   These values signal strong fuel economy. While efficiency often *increases* a car’s value, this vehicle’s **small engine size** (next point) offsets that, making it more affordable.  \n\n2. **Small Engine Size**  \n   - **Engine Size: 92.0** (smaller than typical sedans)  \n   Smaller engines usually mean lower power and **lower cost**. This is the #1 reason the prediction leans toward a moderate price.  \n\n3. **Trusted Brand, Common Engine**  \n   - **Make: Toyota** (reputable brand, holds value well)  \n   - **Engine Type: OHC** (a common, reliable design)  \n   Toyota’s reliability adds value, but the OHC engine is standard—not a

***REMARKS***

I did not perform tests using the eda_findings, although I believe that, following the analysis performed, drawing insights from EDA could provide valuable context for the language model. For example, examining scatter plots or the distribution of specific features could highlight important patterns and relationships that would enrich the model’s understanding.

In addition, I should have experimented more with the temperature parameter, as it controls the degree of creativity or randomness in the model’s responses. This can be particular useful and interesting in the feature engineering agent.